<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Visualización de Datos</p>
<p style="margin: 0; text-align:right;">Máster Universitario de Ciencia de datos </p>
<p style="margin: 0; text-align:right;">Universitat Oberta de Catalunya</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Autor: Jorge Alonso Hernández</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Práctica 2: Creación de la visualización y entrega del proyecto

Carga inicial de librerías

In [1]:
#Carga de librerías
import pandas as pd
import numpy as np 
import itertools

from datetime import datetime

Lectura de los datos de ficheros iniciales

In [2]:
data_covid = pd.read_csv("datasets/covid_canarias.csv")
municipios = pd.read_csv("datasets/municipios.csv")
islas = pd.read_csv("datasets/islas.csv")

Visualizamos una muestra de los datos

In [3]:
data_covid.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
0,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
1,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
2,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
3,14/03/2022,TENERIFE,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
4,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife


Realizamos una transformación en los datos de forma que convertiremos los nombres de las islas en mayúsculas al formato real con cada inicial en mayúsculas sólo.

In [4]:
islas['isla'] = islas['isla'].str.title()
data_covid['isla'] = data_covid['isla'].str.title()

Realizaremos una transformación también en los datos de los municipios ya que podemos ver que existe una incoherencia entre los ficheros con los datos de covid y el de los municipios de forma que adaptaremos los nombres de los municipios a los que existen en el fichero de municipios.

In [5]:
#Obtenenmos los nombres de los municipios en el documento de casos de covid
municipios_ori = pd.DataFrame(pd.unique(data_covid['municipio']))
municipios_ori.columns = ['municipio']

#Comprobamos los nombres de los municipios
df_destino = municipios.loc[~municipios['municipio'].isin(pd.unique(data_covid['municipio']))]
df_destino = pd.DataFrame(df_destino['municipio'])
df_origen = municipios_ori.loc[~municipios_ori['municipio'].isin(municipios['municipio'])]

df_origen = df_origen.sort_values(by='municipio')
df_destino = df_destino.sort_values(by='municipio')

df_origen = df_origen.reset_index()
df_destino = df_destino.reset_index()

In [6]:
print(df_origen)

   index              municipio
0     45               El Pinar
1     81           Fuencaliente
2     70      Icod de Los Vinos
3      5              La Laguna
4     24      Puerto de La Cruz
5     87        SIN ESPECIFICAR
6     32  San juan de La Rambla
7     29    Santa María de Guía
8     67            Valsequillo
9     69               Vilaflor


In [7]:
print(df_destino)

   index                            municipio
0     87                El Pinar de El Hierro
1     47             Fuencaliente de La Palma
2     55                    Icod de los Vinos
3     61                    Puerto de la Cruz
4     56           San Cristóbal de La Laguna
5     67                San Juan de la Rambla
6     22  Santa María de Guía de Gran Canaria
7     30          Valsequillo de Gran Canaria
8     85                   Vilaflor de Chasna


In [8]:
#Eliminamos el valor SIN ESPECIFICAR pues es correcto que estos valores no se visualicen en el mapa 
df_origen = df_origen.drop(5).reset_index()

In [9]:
print(df_origen)

   level_0  index              municipio
0        0     45               El Pinar
1        1     81           Fuencaliente
2        2     70      Icod de Los Vinos
3        3      5              La Laguna
4        4     24      Puerto de La Cruz
5        6     32  San juan de La Rambla
6        7     29    Santa María de Guía
7        8     67            Valsequillo
8        9     69               Vilaflor


In [10]:
for i in range(0,len(df_origen)):
    data_covid = data_covid.replace(df_origen['municipio'][i], df_destino['municipio'][i])


Una vez realizadas las transformaciones de los datos inicializaremos la obtención de los conjuntos de datos que nos van a permitir visualizar los resultados. Inicialmente realizamos una separación entre los casos registrados en función del lugar en el que se ha realizado la asignación del caso y los casos en función del lugar de residencia del paciente.

In [11]:
asignacion = data_covid[data_covid['identificacion_caso'] == 'Asignación']
residencia = data_covid[data_covid['identificacion_caso'] == 'Residencia']

In [12]:
asignacion.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
0,14/03/2022,Tenerife,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
1,14/03/2022,Tenerife,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
2,14/03/2022,Tenerife,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
3,14/03/2022,Tenerife,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
4,14/03/2022,Tenerife,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife


In [13]:
residencia.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
282989,14/03/2022,Tenerife,Adeje,Mujer,20 - 29,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282990,14/03/2022,Tenerife,SIN ESPECIFICAR,Mujer,20 - 29,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282991,14/03/2022,Tenerife,Adeje,Mujer,10 - 19,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282992,14/03/2022,Tenerife,SIN ESPECIFICAR,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282993,14/03/2022,Tenerife,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife


Obtenemos los datos de los municipios, islas y provincias por separado de forma que nos ayuden a obtener los datos en función de cada una de estas categorías.

In [14]:
municipios = municipios.merge(islas[['geocode', 'isla', 'provincia', 'poblacion']],  left_on='geocode_isla', right_on='geocode')

In [15]:
df_municipios = pd.DataFrame(municipios['municipio'])
df_islas = pd.DataFrame(islas['isla'])
df_provincias = pd.DataFrame(pd.unique(islas['provincia']))
df_provincias.columns = ['provincia']

In [16]:
df_municipios

,municipio
0,Agaete
1,Agüimes
2,Artenara
3,Arucas
4,Firgas
...,...
83,Tijarafe
84,Villa de Mazo
85,Frontera
86,Valverde


In [17]:
df_islas

,isla
0,El Hierro
1,Fuerteventura
2,Gran Canaria
3,La Gomera
4,La Palma
5,Lanzarote
6,Tenerife


In [18]:
df_provincias

,provincia
0,Santa Cruz de Tenerife
1,Las Palmas


También obtenemos los datos referentes a las capitales de las islas y de las provincias para la visualización de estas en el mapa.

In [19]:
capitales = ['Santa Cruz de Tenerife', 'Santa Cruz de La Palma', 'San Sebastián de La Gomera', 'Valverde', 'Las Palmas de Gran Canaria', 'Puerto del Rosario', 'Arrecife']
capitales_provincia = ['Santa Cruz de Tenerife', 'Las Palmas de Gran Canaria']

df_capitales = municipios.loc[municipios['municipio'].isin(capitales)]
df_capitales = df_capitales.reset_index()

df_capitales_provincia = municipios.loc[municipios['municipio'].isin(capitales_provincia)]
df_capitales_provincia = df_capitales_provincia.reset_index()

In [20]:
df_capitales = df_capitales[['municipio','isla', 'provincia','longitud_capital','latitud_capital', 'poblacion']]
df_capitales_provincia = df_capitales_provincia[['municipio','isla', 'provincia','longitud_capital','latitud_capital']]

In [21]:
df_capitales

,municipio,isla,provincia,longitud_capital,latitud_capital,poblacion
0,Las Palmas de Gran Canaria,Gran Canaria,Las Palmas,-15.428533,28.125058,855521
1,Puerto del Rosario,Fuerteventura,Las Palmas,-13.859694,28.498774,119732
2,Arrecife,Lanzarote,Las Palmas,-13.546719,28.959235,155812
3,Santa Cruz de Tenerife,Tenerife,Santa Cruz de Tenerife,-16.254634,28.469805,928604
4,San Sebastián de La Gomera,La Gomera,Santa Cruz de Tenerife,-17.109706,28.090378,21678
5,Santa Cruz de La Palma,La Palma,Santa Cruz de Tenerife,-17.764744,28.683243,83458
6,Valverde,El Hierro,Santa Cruz de Tenerife,-17.914964,27.806884,11147


In [22]:
df_capitales_provincia

,municipio,isla,provincia,longitud_capital,latitud_capital
0,Las Palmas de Gran Canaria,Gran Canaria,Las Palmas,-15.428533,28.125058
1,Santa Cruz de Tenerife,Tenerife,Santa Cruz de Tenerife,-16.254634,28.469805


Realizaremos ahora la obtención de los datos finales, se obtendrán por un lado los datos de los nuevos casos de cada uno de los días y a la vez se realizará un conteo de los datos de casos que se van acumulando día a día de forma que se tenga un registro total. Para ello realizamos una función que se va a encargar de realizar el conteo de los casos en función de la fecha y del tipo de agrupamiento que puede ser por municipios, islas o provincias. Una vez realizado el conteo por fecha y tipo de agrupación se irá recorriendo el dataframe resultante y se irán trasponiendo los datos de forma que se obtenga una columna de dataframe por fecha con sus resultados asociados.

In [23]:
def conversion(df, tipo_agrupamiento):
    # Dataframes resultantes
    df_result = pd.DataFrame()
    df_result_accumul = pd.DataFrame()
        
    if tipo_agrupamiento == 'municipios':
        df_agrupamientos = df_municipios
        label = 'municipio'
    elif tipo_agrupamiento == 'islas':
        df_agrupamientos = df_islas
        label = 'isla'
    elif tipo_agrupamiento == 'provincias':
        df_agrupamientos = df_provincias
        label = 'provincia'
    else:
        return None
    
    # El método irá recorriendo cada uno de los municipios/islas/provincias y trasponiendo las filas por columnas para obtener el formato deseado
    for index, row in df_agrupamientos.iterrows():
        aux = df[df[label] == row[label]]
        aux = aux.groupby(['fecha_caso',label]).size().reset_index()
        aux.columns=['fecha', label, row[label]]
        aux = aux[['fecha', row[label]]]
        aux['fecha'] = pd.to_datetime(aux['fecha'], dayfirst=True,format='%d/%m/%Y', exact=True)
        #aux['fecha'] = aux['fecha'].dt.strftime('%Y/%m/%d') #Para poder odernar correctamente
        
        #Ordenamos los resultados
        aux = aux.sort_values(by='fecha')
        aux = aux.set_index('fecha')
        aux_acc = aux.cumsum()
        aux = aux.transpose()
        aux_acc = aux_acc.transpose()
        
        df_result = pd.concat([df_result, aux], axis = 0)
        df_result_accumul = pd.concat([df_result_accumul, aux_acc], axis = 0)
    
    #Se realiza una segunda ordenación ya que el método concat de pandas añade al finalRealizamos la obtención de los datos empleando el método definido, obtenemos en primer lugar los datos en función de la asignación del caso. las columnas que no están en un dataframe con valor Nan
    
    # Ordenamos por fecha y cumplimentamos los NaN con el valor anterior, para poder ordenar por fecha tenemos que resetear los índices y estabelcer la fecha como índice de nuevo
    df_result_accumul = df_result_accumul.transpose()
    df_result_accumul = df_result_accumul.reset_index()  
    df_result_accumul = df_result_accumul.sort_values(by=['fecha']) # Se ordena de nuevo para que las fechas que se han añadido con valor NaN
    df_result_accumul['fecha'] = df_result_accumul['fecha'].dt.strftime('%d/%m/%Y')
    df_result_accumul = df_result_accumul.set_index('fecha')
    df_result_accumul[0:1] = df_result_accumul[0:1].fillna(0) # Si los valores NaN están al incio se cumplimentan con 0
    df_result_accumul = df_result_accumul.fillna(method="pad")# Asignamos el valor inmediatamente anterior a los NaN

    df_result_accumul = df_result_accumul.transpose()
    
    #Ordenamos por fecha y cumplimentamos los NaN por 0, Para poder ordenar por fechas ya que da problemas se hace reseteando y estableciendo de nuevo los índices
    df_result = df_result.replace(np.nan, 0)
    df_result = df_result.transpose()
    df_result = df_result.reset_index()  
    df_result = df_result.sort_values(by=['fecha'])
    df_result['fecha'] = df_result['fecha'].dt.strftime('%d/%m/%Y')
    df_result = df_result.set_index('fecha')
    df_result = df_result.transpose()
    
    df_result = df_result.reset_index()
    df_result_accumul = df_result_accumul.reset_index()
    
    return df_result, df_result_accumul

Realizamos la obtención de los datos empleando el método definido, obtenemos en primer lugar los datos en función de la asignación del caso.

In [24]:
data_municipios_asignacion, data_municipios_asignacion_accumul =  conversion(asignacion, 'municipios')
data_islas_asignacion, data_islas_asignacion_accumul = conversion(asignacion, 'islas')
data_provincias_asignacion, data_provincias_asignacion_accumul = conversion(asignacion, 'provincias')

Obtenemos los datos en función de la residencia del paciente.

In [25]:
data_municipios_residencia, data_municipios_residencia_accumul =  conversion(residencia, 'municipios')
data_islas_residencia, data_islas_residencia_accumul = conversion(residencia, 'islas')
data_provincias_residencia, data_provincias_residencia_accumul = conversion(residencia, 'provincias')

Añadimos los datos de isla y provincia a los datos de los municipios para la visualización.

In [26]:
data_municipios_asignacion = data_municipios_asignacion.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')
data_municipios_asignacion_accumul = data_municipios_asignacion_accumul.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')

data_municipios_residencia = data_municipios_residencia.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')
data_municipios_residencia_accumul = data_municipios_residencia_accumul.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')

Realizamos la visualización de los datos obtenidos y que serán visualizados. en primer lugar, los datos en función de la asignación del caso.

Datos por municipios.

In [27]:
#Datos por día
data_municipios_asignacion

,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022,municipio,isla,provincia
0,Agaete,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,2.0,2.0,2.0,3.0,1.0,0.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,6.0,4.0,9.0,5.0,0.0,1.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,3.0,0.0,1.0,0.0,0.0,0.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,...,3.0,2.0,5.0,8.0,9.0,0.0,0.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,3.0,1.0,3.0,0.0,0.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,2.0,12.0,6.0,7.0,0.0,0.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,2.0,4.0,3.0,4.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,0.0,1.0,12.0,0.0,0.0,1.0,1.0,0.0,...,14.0,23.0,15.0,10.0,17.0,0.0,4.0,Valverde,El Hierro,Santa Cruz de Tenerife


In [28]:
#Datos acumulados
data_municipios_asignacion_accumul

,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022,municipio,isla,provincia
0,Agaete,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,379.0,381.0,383.0,385.0,388.0,389.0,389.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,1016.0,1022.0,1026.0,1035.0,1040.0,1040.0,1041.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.0,52.0,52.0,53.0,53.0,53.0,53.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,0.0,1.0,1.0,1.0,1.0,2.0,3.0,5.0,5.0,...,796.0,798.0,803.0,811.0,820.0,820.0,820.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,263.0,266.0,269.0,270.0,273.0,273.0,273.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,76.0,76.0,76.0,76.0,78.0,79.0,79.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,140.0,142.0,154.0,160.0,167.0,167.0,167.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,...,357.0,358.0,359.0,361.0,365.0,368.0,372.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,0.0,1.0,13.0,13.0,13.0,14.0,15.0,15.0,...,1294.0,1317.0,1332.0,1342.0,1359.0,1359.0,1363.0,Valverde,El Hierro,Santa Cruz de Tenerife


Datos por islas.

In [29]:
#Datos por día
data_islas_asignacion

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,El Hierro,0.0,0.0,1.0,13.0,1.0,0.0,1.0,2.0,0.0,...,11.0,9.0,1.0,18.0,27.0,17.0,15.0,23.0,3.0,8.0
1,Fuerteventura,1.0,4.0,12.0,4.0,26.0,16.0,15.0,29.0,30.0,...,37.0,14.0,19.0,19.0,12.0,18.0,25.0,28.0,20.0,4.0
2,Gran Canaria,62.0,63.0,83.0,72.0,99.0,79.0,75.0,148.0,109.0,...,719.0,236.0,172.0,434.0,458.0,456.0,480.0,436.0,243.0,248.0
3,La Gomera,0.0,1.0,0.0,4.0,0.0,0.0,0.0,3.0,5.0,...,23.0,16.0,2.0,42.0,17.0,29.0,36.0,22.0,21.0,13.0
4,La Palma,0.0,3.0,3.0,1.0,4.0,0.0,2.0,9.0,4.0,...,84.0,24.0,9.0,103.0,98.0,100.0,82.0,87.0,44.0,15.0
5,Lanzarote,14.0,32.0,20.0,21.0,21.0,49.0,61.0,25.0,62.0,...,99.0,41.0,54.0,130.0,129.0,26.0,21.0,23.0,14.0,15.0
6,Tenerife,67.0,83.0,49.0,116.0,99.0,106.0,90.0,92.0,73.0,...,847.0,295.0,286.0,800.0,911.0,886.0,795.0,851.0,356.0,254.0


In [30]:
#Datos acumulados
data_islas_asignacion_accumul

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,El Hierro,0.0,0.0,1.0,14.0,15.0,15.0,16.0,18.0,18.0,...,1746.0,1755.0,1756.0,1774.0,1801.0,1818.0,1833.0,1856.0,1859.0,1867.0
1,Fuerteventura,1.0,5.0,17.0,21.0,47.0,63.0,78.0,107.0,137.0,...,13926.0,13940.0,13959.0,13978.0,13990.0,14008.0,14033.0,14061.0,14081.0,14085.0
2,Gran Canaria,62.0,125.0,208.0,280.0,379.0,458.0,533.0,681.0,790.0,...,103485.0,103721.0,103893.0,104327.0,104785.0,105241.0,105721.0,106157.0,106400.0,106648.0
3,La Gomera,0.0,1.0,1.0,5.0,5.0,5.0,5.0,8.0,13.0,...,1682.0,1698.0,1700.0,1742.0,1759.0,1788.0,1824.0,1846.0,1867.0,1880.0
4,La Palma,0.0,3.0,6.0,7.0,11.0,11.0,13.0,22.0,26.0,...,9786.0,9810.0,9819.0,9922.0,10020.0,10120.0,10202.0,10289.0,10333.0,10348.0
5,Lanzarote,14.0,46.0,66.0,87.0,108.0,157.0,218.0,243.0,305.0,...,18854.0,18895.0,18949.0,19079.0,19208.0,19234.0,19255.0,19278.0,19292.0,19307.0
6,Tenerife,67.0,150.0,199.0,315.0,414.0,520.0,610.0,702.0,775.0,...,123420.0,123715.0,124001.0,124801.0,125712.0,126598.0,127393.0,128244.0,128600.0,128854.0


Datos por provincias.

In [31]:
#Datos por día
data_provincias_asignacion

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,Santa Cruz de Tenerife,67,87,53,134,104,106,93,106,82,...,965,344,298,963,1053,1032,928,983,424,290
1,Las Palmas,77,99,115,97,146,144,151,202,201,...,855,291,245,583,599,500,526,487,277,267


In [32]:
#Datos p
data_provincias_asignacion_accumul

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,Santa Cruz de Tenerife,67,154,207,341,445,551,644,750,832,...,136634,136978,137276,138239,139292,140324,141252,142235,142659,142949
1,Las Palmas,77,176,291,388,534,678,829,1031,1232,...,136265,136556,136801,137384,137983,138483,139009,139496,139773,140040


Visualizamos los datos obtenidos en función de la residencia del paciente

Datos por municipios.

In [33]:
#Datos por día
data_municipios_residencia

,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022,municipio,isla,provincia
0,Agaete,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,3.0,4.0,3.0,3.0,1.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,3.0,1.0,0.0,1.0,4.0,0.0,0.0,0.0,...,7.0,13.0,12.0,15.0,10.0,5.0,4.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,2.0,4.0,1.0,1.0,5.0,5.0,3.0,5.0,5.0,...,13.0,12.0,14.0,20.0,14.0,21.0,14.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,7.0,4.0,4.0,2.0,0.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,2.0,1.0,0.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,13.0,5.0,7.0,0.0,0.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,...,8.0,5.0,5.0,3.0,5.0,2.0,5.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,0.0,1.0,11.0,0.0,0.0,1.0,1.0,0.0,...,9.0,19.0,6.0,11.0,11.0,1.0,2.0,Valverde,El Hierro,Santa Cruz de Tenerife


In [34]:
#Datos acumulados
data_municipios_residencia_accumul

,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022,municipio,isla,provincia
0,Agaete,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,576.0,577.0,580.0,584.0,587.0,590.0,591.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,3.0,4.0,4.0,5.0,9.0,9.0,9.0,9.0,...,3782.0,3795.0,3807.0,3822.0,3832.0,3837.0,3841.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,72.0,72.0,72.0,73.0,73.0,73.0,73.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,2.0,6.0,7.0,8.0,13.0,18.0,21.0,26.0,31.0,...,3614.0,3626.0,3640.0,3660.0,3674.0,3695.0,3709.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,782.0,785.0,792.0,796.0,800.0,802.0,802.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,286.0,286.0,286.0,286.0,288.0,289.0,289.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,584.0,588.0,601.0,606.0,613.0,613.0,613.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,0.0,0.0,2.0,3.0,3.0,3.0,3.0,3.0,...,648.0,653.0,658.0,661.0,666.0,668.0,673.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,0.0,1.0,12.0,12.0,12.0,13.0,14.0,14.0,...,822.0,841.0,847.0,858.0,869.0,870.0,872.0,Valverde,El Hierro,Santa Cruz de Tenerife


Datos por islas.

In [35]:
#Datos por día
data_islas_residencia

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,El Hierro,0.0,0.0,1.0,13.0,1.0,0.0,1.0,2.0,0.0,...,11.0,9.0,1.0,18.0,27.0,17.0,15.0,23.0,3.0,8.0
1,Fuerteventura,1.0,4.0,12.0,4.0,26.0,16.0,15.0,29.0,30.0,...,37.0,14.0,19.0,19.0,12.0,18.0,25.0,28.0,20.0,4.0
2,Gran Canaria,62.0,63.0,83.0,72.0,99.0,79.0,75.0,148.0,109.0,...,719.0,236.0,172.0,434.0,458.0,456.0,480.0,436.0,243.0,248.0
3,La Gomera,0.0,1.0,0.0,4.0,0.0,0.0,0.0,3.0,5.0,...,23.0,16.0,2.0,42.0,17.0,29.0,36.0,22.0,21.0,13.0
4,La Palma,0.0,3.0,3.0,1.0,4.0,0.0,2.0,9.0,4.0,...,84.0,24.0,9.0,103.0,98.0,100.0,82.0,87.0,44.0,15.0
5,Lanzarote,14.0,32.0,20.0,21.0,21.0,49.0,61.0,25.0,62.0,...,99.0,41.0,54.0,130.0,129.0,26.0,21.0,23.0,14.0,15.0
6,Tenerife,67.0,83.0,49.0,116.0,99.0,106.0,90.0,92.0,73.0,...,847.0,295.0,286.0,800.0,911.0,886.0,795.0,851.0,356.0,254.0


In [36]:
#Datos acumulados
data_islas_residencia_accumul

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,El Hierro,0.0,0.0,1.0,14.0,15.0,15.0,16.0,18.0,18.0,...,1746.0,1755.0,1756.0,1774.0,1801.0,1818.0,1833.0,1856.0,1859.0,1867.0
1,Fuerteventura,1.0,5.0,17.0,21.0,47.0,63.0,78.0,107.0,137.0,...,13926.0,13940.0,13959.0,13978.0,13990.0,14008.0,14033.0,14061.0,14081.0,14085.0
2,Gran Canaria,62.0,125.0,208.0,280.0,379.0,458.0,533.0,681.0,790.0,...,103485.0,103721.0,103893.0,104327.0,104785.0,105241.0,105721.0,106157.0,106400.0,106648.0
3,La Gomera,0.0,1.0,1.0,5.0,5.0,5.0,5.0,8.0,13.0,...,1682.0,1698.0,1700.0,1742.0,1759.0,1788.0,1824.0,1846.0,1867.0,1880.0
4,La Palma,0.0,3.0,6.0,7.0,11.0,11.0,13.0,22.0,26.0,...,9786.0,9810.0,9819.0,9922.0,10020.0,10120.0,10202.0,10289.0,10333.0,10348.0
5,Lanzarote,14.0,46.0,66.0,87.0,108.0,157.0,218.0,243.0,305.0,...,18854.0,18895.0,18949.0,19079.0,19208.0,19234.0,19255.0,19278.0,19292.0,19307.0
6,Tenerife,67.0,150.0,199.0,315.0,414.0,520.0,610.0,702.0,775.0,...,123420.0,123715.0,124001.0,124801.0,125712.0,126598.0,127393.0,128244.0,128600.0,128854.0


Datos por provincias.

In [37]:
#Datos por día
data_provincias_residencia

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,Santa Cruz de Tenerife,67,87,53,134,104,106,93,106,82,...,965,344,298,963,1053,1032,928,983,424,290
1,Las Palmas,77,99,115,97,146,144,151,202,201,...,855,291,245,583,599,500,526,487,277,267


In [38]:
#Datos acumulados
data_provincias_residencia_accumul

fecha,index,01/01/2021,02/01/2021,03/01/2021,04/01/2021,05/01/2021,06/01/2021,07/01/2021,08/01/2021,09/01/2021,...,04/03/2022,05/03/2022,06/03/2022,07/03/2022,08/03/2022,09/03/2022,10/03/2022,11/03/2022,12/03/2022,13/03/2022
0,Santa Cruz de Tenerife,67,154,207,341,445,551,644,750,832,...,136634,136978,137276,138239,139292,140324,141252,142235,142659,142949
1,Las Palmas,77,176,291,388,534,678,829,1031,1232,...,136265,136556,136801,137384,137983,138483,139009,139496,139773,140040


Una vez obtenidos los datos realizamos la escritura en formato Excel para realizar la subida a la plataforma de Flourish.

Escritura de los datos en función de la asignación del caso.

In [39]:
data_provincias_asignacion.to_excel('data/asignacion/provincias_asignacion.xlsx', index=False, encoding="utf8")
data_provincias_asignacion_accumul.to_excel('data/asignacion/provincias_asignacion_acc.xlsx', index=False, encoding="utf8")
data_islas_asignacion.to_excel('data/asignacion/islas_asignacion.xlsx', index=False, encoding="utf8")
data_islas_asignacion_accumul.to_excel('data/asignacion/islas_asignacion_acc.xlsx', index=False, encoding="utf8")
data_municipios_asignacion.to_excel('data/asignacion/municipios_asignacion.xlsx', index=False, encoding="utf8")
data_municipios_asignacion_accumul.to_excel('data/asignacion/municipios_asignacion_acc.xlsx', index=False, encoding="utf8")

Escritura de los datos en función de la residencia del paciente.

In [40]:
data_provincias_residencia.to_excel('data/residencia/provincias_residencia.xlsx', index=False, encoding="utf8")
data_provincias_residencia_accumul.to_excel('data/residencia/provincias_residencia_acc.xlsx', index=False, encoding="utf8")
data_islas_residencia.to_excel('data/residencia/islas_residencia.xlsx', index=False, encoding="utf8")
data_islas_residencia_accumul.to_excel('data/residencia/islas_residencia_acc.xlsx', index=False, encoding="utf8")
data_municipios_residencia.to_excel('data/residencia/municipios_residencia.xlsx', index=False, encoding="utf8")
data_municipios_residencia_accumul.to_excel('data/residencia/municipios_residencia_acc.xlsx', index=False, encoding="utf8")

Por último realizamos la escritura de los datos de las capitales.

In [41]:
df_capitales.to_csv('mapas/capitales.csv', index=False, sep=",")
df_capitales_provincia.to_csv('mapas/capitales_provincia.csv', index=False, sep=",")